In [1]:
import itertools
import json
import os

import numpy as np
from pathlib import Path
import pandas as pd

In [2]:
dataset="cs_fleurs"
BASE_DIR = f"../../evaluation/output_evals/{dataset}"
DIRECTION_PAIRS = [f'{x}_en' for x in ("es","de","fr","zh")]
DIRECTION_PAIRS

['es_en', 'de_en', 'fr_en', 'zh_en']

In [3]:
SYSTEM_NAMES = os.listdir(BASE_DIR)
sorted(SYSTEM_NAMES)
SYSTEM_NAMES = SYSTEM_NAMES + ["spirellm"]

In [ ]:
def load_results_summaries(base_dir, direction_pairs, system_names):
    """
    Loads all result summaries from a directory structure.

    Args:
        base_dir (str or Path): The base directory for the evaluation outputs.
        direction_pairs (list): A list of language direction strings (e.g., 'en_de').
        system_names (list): A list of system name strings.

    Returns:
        dict: A nested dictionary containing the loaded data, structured as
              {direction: {system: [results]}}.
    """
    base_path = Path(base_dir)
    all_results = {}

    # Use itertools.product to cleanly iterate over all combinations
    for direction, system in itertools.product(direction_pairs, system_names):
        summary_path = base_path / system / direction / 'results.jsonl'
        
        # Initialize the nested dictionary structure
        if direction not in all_results:
            all_results[direction] = {}

        try:
            with summary_path.open('r', encoding='utf-8') as f:
                all_results[direction][system] = [json.loads(line) for line in f]
                
        except FileNotFoundError:
            print(f"Warning: File not found, skipping: {summary_path}")
            all_results[direction][system] = None # Or [] if you prefer an empty list
        except json.JSONDecodeError as e:
            print(f"Error decoding JSON in {summary_path}: {e}")
            all_results[direction][system] = None

    return all_results

In [5]:
def convert_results_to_dataframe(results_data):
    """
    Converts the nested dictionary of results into a single pandas DataFrame.

    Each row corresponds to a single entry, with 'direction' and 'system'
    columns added, and all 'metrics' unpacked into separate columns.
    """
    all_records = []
    for direction, systems in results_data.items():
        for system, records in systems.items():
            if records is None:
                continue
            for record in records:
                # Separate metrics from the record
                metrics = record.pop("metrics", {})  # safely get metrics
                # Merge everything into one flat dict
                flat_record = {
                    "direction": direction,
                    "system": system,
                    **record,
                    **metrics,  # unpack metrics into top-level keys
                }
                all_records.append(flat_record)

    if not all_records:
        print("No records were found to create a DataFrame.")
        return pd.DataFrame()

    df = pd.DataFrame(all_records)

    # Put identifying info up front
    original_cols = [c for c in df.columns if c not in ["direction", "system"]]
    df = df[["direction", "system"] + original_cols]

    return df

In [7]:
def compute_strict_scores(df):
    """
    Computes mean metric scores and strict scores grouped by (system, accent).
    
    Expects columns:
      - system
      - accent
      - xcomet_qe_score
      - metricx_qe_score
      - linguapy_score (list/tuple of [flag, lang])
    """
    df = df.copy()

    # --- Split linguapy_score into two separate columns ---
    df[["linguapy_flag", "linguapy_lang"]] = pd.DataFrame(
        df["linguapy_score"].tolist(), index=df.index
    )

    # --- Define penalties ---
    penalty_by_metric = {
        "metricx_qe": 25,
        "xcomet_qe": 0,
    }

    # --- Strict score per row ---
    for metric in penalty_by_metric.keys():
        df[f"{metric}_strict"] = df.apply(
            lambda row: row[f"{metric}_score"]
            if row["linguapy_flag"] == 0
            else penalty_by_metric[metric],
            axis=1,
        )

    # --- Aggregate by system × accent ---
    agg_cols = {
        "linguapy_flag": "mean",  # average from 0–1
    }
    for metric in penalty_by_metric.keys():
        agg_cols[f"{metric}_score"] = "mean"
        agg_cols[f"{metric}_strict"] = "mean"

    result = (
        df.groupby(["system"])
        .agg(agg_cols)
        .reset_index()
        .rename(columns={"linguapy_flag": "linguapy_avg"})
    )

    result['linguapy_avg'] = result['linguapy_avg']*100
    result["metricx_qe_score"] = 100-4*result["metricx_qe_score"] 
    result["metricx_qe_strict"] = 100-4*result["metricx_qe_strict"] 
    return result, df

In [8]:
results_full = load_results_summaries(BASE_DIR, DIRECTION_PAIRS, SYSTEM_NAMES)

In [9]:
df = convert_results_to_dataframe(results_full)
df.head(3)

,direction,system,dataset_id,sample_id,src_lang,tgt_lang,output,bleu_score,chrf_score,xcomet_score,xcomet_qe_score,metricx_score,metricx_qe_score,linguapy_score
0,es_en,phi4multimodal,cs_fleurs,0,es,en,Hydrogen ions are protons that are stripped of...,20.187382,63.558024,0.987867,0.999209,1.119400,0.807129,"[0, ENGLISH]"
1,es_en,phi4multimodal,cs_fleurs,1,es,en,Large television screens were installed in var...,59.088710,82.233391,0.986656,1.000000,1.739823,1.991402,"[0, ENGLISH]"
2,es_en,phi4multimodal,cs_fleurs,2,es,en,Casablanca is one of the least interesting pla...,62.628450,87.860942,1.000000,1.000000,0.898316,1.081494,"[0, ENGLISH]"


In [ ]:
col_map = {
    "linguapy_avg":"LinguaPy",
    "metricx_qe_strict":"QEMetricX_24-Strict-linguapy",
    "xcomet_qe_strict": "XCOMET-QE-Strict-linguapy"
}
import seaborn as sns
import matplotlib.pyplot as plt

#Collapse and get the metrics balanced by the linguapy score
for pair in DIRECTION_PAIRS:
    sub_df = df[df['direction']==pair]
    sub_df, df_sentence = compute_strict_scores(sub_df)
    sub_df = sub_df.rename(columns=col_map)
    sub_df.loc[len(sub_df)+1] = None
    sub_df.loc[len(sub_df), "system"] = "spirelm"
    sub_df.to_csv(f"{dataset}_{pair}.csv",index=False)

In [16]:
!python3 combine_csv.py -i cs_fleurs*.csv -oc combined.csv -ot cs-fleurs.tex

dict_keys(['de-en', 'es-en', 'fr-en', 'zh-en'])
['es-en', 'de-en', 'fr-en', 'zh-en']
